# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

pip install notebook ipykernel
pip install gradio requests

In [ ]:
import gradio as gr 


In [ ]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
# Adding share=True means that it can be accessed publically
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Adding authentication

Gradio makes it very easy to have userids and passwords

Obviously if you use this, have it look properly in a secure place for passwords! At a minimum, use your .env

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("mob", "12345"))

## changing the theme


In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface


gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", theme=gr.themes.Glass()).launch()

In [ ]:
# Adding a little more:

message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
import gradio as gr

def process(text, number):
    shouted = text.upper() + "!!!"
    length = len(text)
    squared = number * number
    return shouted, length, squared

# Create input components
text_input = gr.Textbox(label="Enter Text")
number_input = gr.Number(label="Enter a Number")

# Create output components
shout_output = gr.Textbox(label="Shouted Text")
length_output = gr.Number(label="Text Length")
square_output = gr.Number(label="Number Squared")

# Create Interface
demo = gr.Interface(
    fn=process,
    inputs=[text_input, number_input],
    outputs=[shout_output, length_output, square_output],
    title="Multiple Inputs & Outputs Example",
    flagging_mode="never"
)

demo.launch()


In [ ]:
def message_ollama(text):
  prompt = f"""
  {text}
  """
  payload = {
    "model" : "gemma3:270m",
    "prompt" : prompt,
    "stream" : False
  }

  response = requests.post("http://127.0.0.1:11434/api/generate", json=payload)
  return (response.json())["response"]

In [ ]:
# And now - changing the function from "shout" to "gemini api"

message_input = gr.Textbox(label="Your message:", info="Enter a message for ollama", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_ollama,
    title="Ollama", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "how are you"], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
# Let's use Markdown

system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for ollama", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_ollama,
    title="Ollama", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

# Building a company brochure generator

Now you know how - it's simple!

In [ ]:
import gradio as gr
from scrapper import fetch_website_contents, fetch_website_links
import json
import requests


# ==============================
# OLLAMA CONFIG
# ==============================

OLLAMA_URL = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "gemma3:270m"   # Change if needed


# ==============================
# LLM CALL (Ollama Only)
# ==============================

def generate_with_ollama(prompt):

    payload = {
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(OLLAMA_URL, json=payload, timeout=120)

    if response.status_code == 200:
        return response.json()["response"]
    else:
        print("Ollama error:", response.text)
        return ""


# ==============================
# STEP 1 – Filter Links
# ==============================

def filter_links(company_url, links):

    links = links[:25]

    prompt = f"""
Help me analyze a company website.

Main website:
{company_url}

These are extracted links:
{links}

Filter only links relevant for a sales brochure:
- About page
- Services
- Products
- Company profile
- Solutions
- Overview

STRICT RULES:
- Return ONLY valid JSON
- No markdown
- No explanation
- No backticks

Example:
["https://company.com/about", "https://company.com/services"]
"""

    res = generate_with_ollama(prompt)
    raw_text = res.strip()

    # Safety cleanup (in case model adds markdown)
    if raw_text.startswith("```"):
        raw_text = raw_text.replace("```json", "").replace("```", "").strip()

    try:
        return json.loads(raw_text)
    except:
        return []


# ==============================
# STEP 2 – Collect Company Data
# ==============================

def collect_company_data(main_url, relevant_links):

    combined_text = fetch_website_contents(main_url)

    for link in relevant_links:
        try:
            combined_text += "\n\n" + fetch_website_contents(link)
        except Exception:
            pass

    return combined_text[:6000]  # token safety


# ==============================
# STEP 3 – Generate Brochure
# ==============================

def generate_sales_brochure(company_data):

    prompt = f"""
You are a professional marketing copywriter.

Using the information below, create a clear and
persuasive sales brochure for potential clients
or investors.

Include:
- Company overview
- Products / services
- Unique value proposition
- Target customers
- Why choose this company

Company information:
{company_data}
"""

    return generate_with_ollama(prompt)


# ==============================
# MAIN ORCHESTRATION
# ==============================

def fetch_brochure(company_url):

    print("Fetching links...")
    links = fetch_website_links(company_url)

    print("Filtering relevant links with Ollama...")
    relevant_links = filter_links(company_url, links)

    print("Collecting company data...")
    company_data = collect_company_data(company_url, relevant_links)

    print("Generating sales brochure...\n")
    brochure = generate_sales_brochure(company_data)

    print("========== SALES BROCHURE ==========\n")
    return brochure


# ==============================
# GRADIO UI
# ==============================

message_input = gr.Textbox(
    label="Your Company URL:",
    info="Enter a full company URL",
    lines=1
)

message_output = gr.Markdown(label="Sales Brochure")

view = gr.Interface(
    fn=fetch_brochure,
    title="AI Sales Brochure Generator (Ollama)",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        ["https://www.its.edu.in/"],
        ["https://www.openai.com"],
    ],
    flagging_mode="never"
)

view.launch()
